## Part 1 File Processing

In [9]:
import os
import zipfile
import io
import csv
import pandas as pd
import numpy as np
from google.cloud import bigquery
from pandas_gbq import to_gbq
import re


# Define paths
data_directory = 'data/WedgeZipOfZips'  
zip_files = 'data/WedgeZipOfZips'

# Create directory for unzipped files if it doesn't exist
##if not os.path.exists(full_unzip_folder):
#    os.makedirs(full_unzip_folder)

# Iterate through all zip files in the zip folder
#for filename in os.listdir(zip_folder):
    #if filename.endswith('.zip'):
        #zip_file_path = os.path.join(zip_folder, filename)
        #with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
            #zip_ref.extractall(full_unzip_folder)

#print(f"All zip files have been extracted to {full_unzip_folder}")

    

In [10]:
# Schema definition mapping (based on the BigQuery schema)
schema = {
    'datetime': 'datetime64[ns]',
    'register_no': 'float',
    'emp_no': 'float',
    'trans_no': 'float',
    'upc': 'string',
    'description': 'string',
    'trans_type': 'string',
    'trans_subtype': 'string',
    'trans_status': 'string',
    'department': 'float',
    'quantity': 'float',
    'Scale': 'float',
    'cost': 'float',
    'unitPrice': 'float',
    'total': 'float',
    'regPrice': 'float',
    'altPrice': 'float',
    'tax': 'float',
    'taxexempt': 'float',
    'foodstamp': 'float',
    'wicable': 'float',
    'discount': 'float',
    'memDiscount': 'float',
    'discountable': 'float',
    'discounttype': 'float',
    'voided': 'float',
    'percentDiscount': 'float',
    'ItemQtty': 'float',
    'volDiscType': 'float',
    'volume': 'float',
    'VolSpecial': 'float',
    'mixMatch': 'float',
    'matched': 'float',
    'memType': 'bool',
    'staff': 'bool',
    'numflag': 'float',
    'itemstatus': 'float',
    'tenderstatus': 'float',
    'charflag': 'string',
    'varflag': 'float',
    'batchHeaderID': 'bool',
    'local': 'float',
    'organic': 'float',
    'display': 'bool',
    'receipt': 'float',
    'card_no': 'float',
    'store': 'float',
    'branch': 'float',
    'match_id': 'float',
    'trans_id': 'float'
}

In [11]:
# Define the correct headers
correct_headers = [
    'datetime', 'register_no', 'emp_no', 'trans_no', 'upc', 'description', 'trans_type',
    'trans_subtype', 'trans_status', 'department', 'quantity', 'Scale', 'cost',
    'unitPrice', 'total', 'regPrice', 'altPrice', 'tax', 'taxexempt', 'foodstamp',
    'wicable', 'discount', 'memDiscount', 'discountable', 'discounttype', 'voided',
    'percentDiscount', 'ItemQtty', 'volDiscType', 'volume', 'VolSpecial', 'mixMatch',
    'matched', 'memType', 'staff', 'numflag', 'itemstatus', 'tenderstatus', 'charflag',
    'varflag', 'batchHeaderID', 'local', 'organic', 'display', 'receipt', 'card_no',
    'store', 'branch', 'match_id', 'trans_id'
]


In [12]:
# Schema Casting
def cast_dataframe_to_schema(df, schema):
    print(f"Casting DataFrame to BigQuery schema")
    try:
        for field in schema:
            column_name = field.name
            field_type = field.field_type
 
            if column_name in df.columns:
                if field_type == "STRING":
                    df[column_name] = df[column_name].astype(str)
                elif field_type == "FLOAT":
                    df[column_name] = pd.to_numeric(df[column_name], errors='coerce')  # Coerce invalid values to NaN
                elif field_type == "BOOLEAN":
                    df[column_name] = df[column_name].apply(lambda x: bool(x) if pd.notnull(x) else None)  # Handle nulls
                elif field_type == "TIMESTAMP":
                    df[column_name] = pd.to_datetime(df[column_name], errors='coerce', utc=True)  # Convert to datetime and coerce errors
 
        df = df[[field.name for field in schema if field.name in df.columns]]
        return df
    except Exception as e:
        print(f"Error casting DataFrame to schema: {e}")
        raise

In [13]:
# Directory to save processed files
processed_files_directory = "data/processed_files/"

#Process the zip files
zip_files = os.listdir(data_directory)

In [35]:
# Create directory for processed files if it doesn't exist
if not os.path.exists(processed_files_directory):
    os.makedirs(processed_files_directory)

# Process CSV files (since they are already unzipped)
csv_files = os.listdir(data_directory)

# Iterate over each CSV file
for file_name in csv_files:
    if file_name.endswith('.csv'):  # Only process CSV files
        csv_file_path = os.path.join(data_directory, file_name)
        
        with open(csv_file_path, 'r', encoding="utf-8") as working_file:
            try:
                # Take a sample of the file to find headers and delimiters
                sample = working_file.read(1024 * 4)  # Sample for sniffing

                # Find Delimiters and Headers
                sniffer = csv.Sniffer()
                dialect = sniffer.sniff(sample, delimiters=[",", ";", "\t"])
                has_header = sniffer.has_header(sample)
                print(f"For {file_name}, the delimiter is '{dialect.delimiter}' and has header: {has_header}")

                # Reset the file pointer to read again from the start
                working_file.seek(0)

                # Load the CSV
                df = pd.read_csv(
                    csv_file_path,
                    delimiter=dialect.delimiter,
                    header=0 if has_header else None,
                    low_memory=False
                )

                # If the file doesn't have headers, assign the correct headers
                if not has_header:
                    df.columns = correct_headers
                    print(f"Assigned correct headers to {file_name}.")

                # Parse the datetime column, if it exists
                if 'datetime' in df.columns:
                    print(f"Parsing 'datetime' column for {file_name}...")
                    df['datetime'] = pd.to_datetime(df['datetime'], errors='coerce')

                print(f"Loaded {file_name} successfully!")

            except Exception as e:
                print(f"Error loading {file_name}: {e}")
                continue  # Move to the next file if there's an error

            # Ensure df is not None and has data
            if df is not None and not df.empty:
                try:
                    # Replace various null representations with a uniform 'NULL'
                    df = df.replace({
                        '/N': 'NULL',
                        '//N': 'NULL',
                        ' ': 'NULL',
                        '': 'NULL',
                        'NaN': 'NULL',
                        'nan': 'NULL',
                        'N': 'NULL',
                        np.nan: 'NULL'
                    })
                    
                    # Preprocess: Replace any remaining instances of \N with an empty string for BigQuery
                    df.replace(r'\\N', '', inplace=True)

                    print(f"Missing Values and '\\N' converted to acceptable values for {file_name}")

                    # Save the processed DataFrame
                    processed_filename = os.path.join(
                        processed_files_directory, f"{os.path.splitext(file_name)[0]}_processed.csv"
                    )
                    df.to_csv(processed_filename, index=False)
                    print(f"Processed and saved {file_name}.")

                except Exception as e:
                    print(f"Error processing {file_name}: {e}")

    print(f"Completed processing file: {file_name}")


For transArchive_201207_201209_inactive_processed_casted.csv, the delimiter is ',' and has header: True
Parsing 'datetime' column for transArchive_201207_201209_inactive_processed_casted.csv...
Loaded transArchive_201207_201209_inactive_processed_casted.csv successfully!
Missing Values and '\N' converted to acceptable values for transArchive_201207_201209_inactive_processed_casted.csv
Processed and saved transArchive_201207_201209_inactive_processed_casted.csv.
Completed processing file: transArchive_201207_201209_inactive_processed_casted.csv
For transArchive_201611_processed.csv, the delimiter is ',' and has header: True
Parsing 'datetime' column for transArchive_201611_processed.csv...
Loaded transArchive_201611_processed.csv successfully!
Missing Values and '\N' converted to acceptable values for transArchive_201611_processed.csv
Processed and saved transArchive_201611_processed.csv.
Completed processing file: transArchive_201611_processed.csv
For transArchive_201606_processed.csv,

## Part 2 Upload Plan

1. Get all of the file names
1. Read in each file one at a time
1. Define table name
1. Upload data


In [36]:
# Configure GBQ project

project_id = 'juliehilley'
dataset_id = 'wedge_A'

#client = bigquery.Client(project = project_id)
#credentials = client._credentials

In [37]:
# Initialize client
client = bigquery.Client(project="juliehilley")

# Define dataset
dataset_id = f"{client.project}.wedge_A"
dataset = bigquery.Dataset(dataset_id)

# Create the dataset
dataset.location = "US"  # Set location, adjust if needed
dataset = client.create_dataset(dataset, exists_ok=True)
print(f"Created dataset {client.project}.wedge_A")

Created dataset juliehilley.wedge_A


In [38]:
# Set data path
data_directory = "data/processed_files/"

In [39]:
# GBQ schema
schema = [
    bigquery.SchemaField("datetime", "TIMESTAMP", mode="NULLABLE"),
    bigquery.SchemaField("register_no", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("emp_no", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("trans_no", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("upc", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("description", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("trans_type", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("trans_subtype", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("trans_status", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("department", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("quantity", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("Scale", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("cost", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("unitPrice", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("total", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("regPrice", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("altPrice", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("tax", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("taxexempt", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("foodstamp", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("wicable", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("discount", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("memDiscount", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("discountable", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("discounttype", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("voided", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("percentDiscount", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("ItemQtty", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("volDiscType", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("volume", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("VolSpecial", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("mixMatch", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("matched", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("memType", "BOOLEAN", mode="NULLABLE"),
    bigquery.SchemaField("staff", "BOOLEAN", mode="NULLABLE"),
    bigquery.SchemaField("numflag", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("itemstatus", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("tenderstatus", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("charflag", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("varflag", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("batchHeaderID", "BOOLEAN", mode="NULLABLE"),
    bigquery.SchemaField("local", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("organic", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("display", "BOOLEAN", mode="NULLABLE"),
    bigquery.SchemaField("receipt", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("card_no", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("store", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("branch", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("match_id", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("trans_id", "FLOAT", mode="NULLABLE")
]

# Set the schema in the job configuration
job_config = bigquery.LoadJobConfig(schema=schema)

In [40]:
# Configure GBQ
job_config = bigquery.LoadJobConfig(
    schema=schema,
    source_format=bigquery.SourceFormat.CSV,
    skip_leading_rows=1,  #skip header row
    write_disposition = bigquery.WriteDisposition.WRITE_APPEND,
    schema_update_options = [bigquery.SchemaUpdateOption.ALLOW_FIELD_ADDITION]
)

In [41]:
# Set up data dictionaries
clean_files = os.listdir(data_directory)
file_ending = "_processed.csv"

In [42]:
for file_name in os.listdir(data_directory):
    if file_name.endswith("_processed.csv"):
        # Construct full path and table name
        full_path = os.path.join(data_directory, file_name)
        table_name = file_name.replace("_processed.csv", "")  # Remove suffix
        full_table_name = f"{dataset_id}.{table_name}"  # Full table name

        # Check if the file exists
        if os.path.exists(full_path):
            print(f"Uploading {file_name} to BigQuery...")

            # Open the CSV and upload to BigQuery
            with open(full_path, 'rb') as source_file:
                job = client.load_table_from_file(
                    source_file,
                    full_table_name,
                    job_config=job_config,
                )
                job.result()  # Wait for the job to complete
                print(f"{file_name} uploaded successfully to {full_table_name}.")
        
        else:
            print(f"File not found: {file_name}")

Uploading transArchive_201310_201312_processed_processed.csv to BigQuery...


BadRequest: 400 Error while reading data, error message: CSV table encountered too many errors, giving up. Rows: 100; errors: 100. Please look into the errors[] collection for more details.; reason: invalid, message: Error while reading data, error message: CSV table encountered too many errors, giving up. Rows: 100; errors: 100. Please look into the errors[] collection for more details.; reason: invalid, message: Error while reading data, error message: CSV processing encountered too many errors, giving up. Rows: 100; errors: 100; max bad: 0; error percent: 0; reason: invalid, message: Error while reading data, error message: Unable to parse; line_number: 2 byte_offset_to_start_of_line: 460 column_index: 33 column_name: "memType" column_type: BOOL value: "NULL"; reason: invalid, message: Error while reading data, error message: Unable to parse; line_number: 3 byte_offset_to_start_of_line: 684 column_index: 33 column_name: "memType" column_type: BOOL value: "NULL"; reason: invalid, mes